# 第3章: 正規表現
Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．
1行に1記事の情報がJSON形式で格納される
各行には記事名が"title"キーに，記事本文が"text"キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
ファイル全体はgzipで圧縮される
以下の処理を行うプログラムを作成せよ．

##### 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [1]:
import json

contory_json_data = ""
with open("jawiki-country.json", "r", encoding="utf-8") as contory_json_file:
    for line in contory_json_file:
        json_line = json.loads(line)
        if json_line["title"] == u"イギリス" :
            contory_json_data = json_line["text"]
            break;

##### 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [2]:
import re

regex_category_line = '\[\[Category:[^\]]+\]\]'
category_list = re.findall(regex_category_line, contory_json_data)
#print(category_list)

##### 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [3]:
regex_category_name = '\[\[Category:([^\]]+)\]\]'
category_name_list = re.findall(regex_category_name, contory_json_data)
#print(category_name_list)

##### 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

In [4]:
regex_session = '\n(={2,5}[^=]+={2,5})\n'
session_list = re.findall(regex_session, contory_json_data)

session_level = {4: 1, 6: 2, 8: 3, 10: 4}

session_level_list = list()
for session in session_list:
    level = session_level[session.count("=")]
    session_level_list.append("{}.{}".format(str(level).rjust(level,' '), session.replace("=", "")))

#print(session_level_list)

##### 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [5]:
regex_file_name = '\[*[ファイル,File]:([^\|]+)'
file_name_list = re.findall(regex_file_name, contory_json_data)
#print(file_name_list)

##### 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．
##### 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．
##### 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．
##### 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [6]:
regex_template = '\{\{基礎情報(.+)\n\}\}\n'
template_list = re.findall(regex_template, contory_json_data, flags=(re.MULTILINE | re.DOTALL))
template_list = template_list[0].split('\n|')

template_dict = {}
for line in template_list:
    temp_list = line.split(" = ")
    if len(temp_list) > 1:
        # 26,27,28
        template_dict[temp_list[0]] = re.sub("\'{2,5}|\[\[|\]\]|<.+>","",temp_list[1])

#print(template_dict)

##### 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [7]:
import urllib.parse
import urllib.request

file_name = template_dict["国旗画像"]
url = 'https://www.mediawiki.org/w/api.php?' \
    + 'action=query' \
    + '&titles=File:' + urllib.parse.quote(file_name) \
    + '&format=json' \
    + '&prop=imageinfo' \
    + '&iiprop=url'

req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11"})
rep_json_data = json.loads(urllib.request.urlopen(req).read().decode())
flag_url = rep_json_data['query']['pages']['-1']['imageinfo'][0]['url']

print(flag_url)

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
